In [1]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

In [2]:
import basic_functions as bf
import extended_functions as ef
import training_functions as tf
import torch
from models import *
import time

In [3]:
# Initialize NN variables
hidden_sizes_v1 = [1000, 512, 256, 128, 7]
hidden_sizes_v2 = [1000, 256, 64, 7]
hidden_sizes_v3 = [1000, 128, 7]
hidden_sizes_v4 = [1000, 256, 256, 7]
hidden_sizes = [hidden_sizes_v1, hidden_sizes_v2, hidden_sizes_v3, hidden_sizes_v4]

criterion = nn.MSELoss(reduction='mean')

Bmods  = [BasicMLP(hidden_size) for hidden_size in hidden_sizes]
E1mods = [ExtendedMLP_v1(hidden_size) for hidden_size in hidden_sizes]
E2mods = [ExtendedMLP_v2(hidden_size) for hidden_size in hidden_sizes]
Smods  = [SuperExtendedMLP(hidden_sizes_v1)]

Bparams  = [tf.load_model("basic"+str(idx+1).zfill(3)) for idx in range(4)]
E1params = [tf.load_model(("extd"+str(idx+1).zfill(3))+"_v1") for idx in range(4)]
E2params = [tf.load_model(("extd"+str(idx+1).zfill(3))+"_v2") for idx in range(4)]
Sparams  = [tf.load_model("spex"+str(1).zfill(3))]

for idx in range(4):
    Bmods[idx].load_state_dict(Bparams[idx])
    E1mods[idx].load_state_dict(E1params[idx])
    E2mods[idx].load_state_dict(E2params[idx])
Smods[0].load_state_dict(Sparams[0])

Bopts = [torch.optim.Adam(model.parameters()) for model in Bmods]
E1opts = [torch.optim.Adam(model.parameters()) for model in E1mods]
E2opts = [torch.optim.Adam(model.parameters()) for model in E2mods]
Sopts = [torch.optim.Adam(model.parameters()) for model in Smods]

In [4]:
# Data loaders
tn_lder_basic, tt_lder_basic = bf.get_data(0.8)
tn_lder_extdd, tt_lder_extdd = ef.get_data(0.8)

In [12]:
# Train the basic models
for idx,(model, optimizer) in enumerate(zip(Bmods, Bopts)):
    model_name = "basic"+str(idx+1).zfill(3)
    print("----------------------------------------------------")
    print("Testing model: ", model_name)
    start_time = time.time()
    tf.test_model(model, criterion, tt_lder_basic)
    print("Approx Query Time:", (time.time()-start_time)/len(tt_lder_basic.dataset))

----------------------------------------------------
Testing model:  basic001
Test Loss: 0.5313
Example:
-------------------------------------------------------------
Output: [ 4.142  2.099  1.156  0.166  0.977  0.000  0.135]
Target: [ 4.153  2.103  1.136  0.108  0.654  0.741  0.107]
-------------------------------------------------------------
-------------------------------------------------------------
Output: [ 0.000  0.777  1.125  0.000  1.000  0.000  0.000]
Target: [-0.324  0.771  1.127  0.178  0.644  0.732  0.135]
-------------------------------------------------------------
-------------------------------------------------------------
Output: [ 1.653  0.418  0.996  0.000  1.000  0.000  0.000]
Target: [ 1.654  0.421  0.983 -0.029  0.667  0.744 -0.034]
-------------------------------------------------------------
Approx Query Time: 0.00019656406359726123
----------------------------------------------------
Testing model:  basic002
Test Loss: 0.4768
Example:
----------------------

In [11]:
# Train the extended models
for idx,(model, optimizer) in enumerate(zip(E1mods, E1opts)):
    model_name = "extd"+str(idx+1).zfill(3)+"_v1"
    print("----------------------------------------------------")
    print("Testing model: ", model_name)
    start_time = time.time()
    tf.test_model(model, criterion, tt_lder_extdd)
    print("Approx Query Time:", (time.time()-start_time)/len(tt_lder_extdd.dataset))

for idx,(model, optimizer) in enumerate(zip(E2mods, E2opts)):
    model_name = "extd"+str(idx+1).zfill(3)+"_v2"
    print("-------------------------------------------------------------")
    print("Testing model: ", model_name)
    start_time = time.time()
    tf.test_model(model, criterion, tt_lder_extdd)
    print("Approx Query Time:", (time.time()-start_time)/len(tt_lder_extdd.dataset))

for idx,(model, optimizer) in enumerate(zip(Smods, Sopts)):
    model_name = "spex"+str(idx+1).zfill(3)
    print("-------------------------------------------------------------")
    print("Testing model: ", model_name)
    start_time = time.time()
    tf.test_model(model, criterion, tt_lder_extdd)
    print("Approx Query Time:", (time.time()-start_time)/len(tt_lder_extdd.dataset))

----------------------------------------------------
Testing model:  extd001_v1
Test Loss: 0.5275
Example:
----------------------------------------------------
Output: [ 2.286  1.684  1.578  0.000  0.670  0.742  0.000]
Target: [ 2.170  1.617  1.536 -0.356  0.601  0.626 -0.347]
----------------------------------------------------
----------------------------------------------------
Output: [ 0.818  1.763  1.271  0.000  0.622  0.783  0.000]
Target: [ 0.765  1.702  1.204 -0.143  0.628  0.751 -0.144]
----------------------------------------------------
----------------------------------------------------
Output: [ 2.227  0.000  1.185  0.000  0.502  0.865  0.000]
Target: [ 2.169 -0.271  1.165 -0.228  0.467  0.803 -0.291]
----------------------------------------------------
Approx Query Time: 0.05224418640136719
----------------------------------------------------
Testing model:  extd002_v1
Test Loss: 0.9088
Example:
----------------------------------------------------
Output: [ 3.877  0.000